In [1]:
!pip install sparqlwrapper rdflib owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 35.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.9 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.45-cp310-cp310-linux_x86_64.whl size=24077432 sha256=6cf59029926c00c14c11e75296235b78737bb6fb4c2531416532b1d64431a04d
  Stored in directory: /root/.cache/pip/wheels/5c/f4/9d/249b1671d391e3feddd443c7d0eb79a732afabc9f370591271
Successfully built owlready2


In [2]:
import SPARQLWrapper
import urllib
import rdflib
import requests
import json
import pandas as pd
from IPython.display import display


In [3]:
url = 'https://dl.dropboxusercontent.com/scl/fi/mewoj8qv6dvuma8euvm5q/GM_players_statistics.csv?rlkey=cnacz87zt43kmc3xqejzka8b4&dl=0'

chess_df = pd.read_csv(url)
chess_df.head()


,Unnamed: 0,name,username,profile,user_id,title,fide,country,followers,joined,...,highest_bullet_date,bullet_win,bullet_draw,bullet_loss,highest_tactics_rating,highest_tactics_date,highest_puzzle_rush_score,is_streamer,status,league
0,0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,494,07/11/2015,...,NaN,NaN,NaN,NaN,400,07/11/2015,NaN,False,premium,NaN
1,1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,360,11/08/2017,...,27/09/2021,32.0,8.0,47.0,3251,15/09/2021,49.0,False,premium,Bronze
2,2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,199,05/11/2014,...,NaN,NaN,NaN,NaN,400,05/11/2014,NaN,False,premium,NaN
3,3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,142482,27/08/2010,...,28/03/2023,662.0,98.0,246.0,400,27/08/2010,NaN,False,premium,Elite
4,4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,14445,17/03/2013,...,30/11/2021,713.0,65.0,203.0,3519,14/06/2023,64.0,False,premium,Crystal


In [4]:
chess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1508 non-null   int64  
 1   name                       1241 non-null   object 
 2   username                   1508 non-null   object 
 3   profile                    1508 non-null   object 
 4   user_id                    1508 non-null   int64  
 5   title                      1508 non-null   object 
 6   fide                       913 non-null    float64
 7   country                    1508 non-null   object 
 8   followers                  1508 non-null   int64  
 9   joined                     1508 non-null   object 
 10  last_online                1508 non-null   object 
 11  current_rapid_rating       1055 non-null   float64
 12  highest_rapid_rating       997 non-null    float64
 13  highest_rapid_date         997 non-null    objec

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
print(chess_df.head())

   Unnamed: 0                name        username                                      profile   user_id title    fide         country  followers      joined last_online  current_rapid_rating  highest_rapid_rating highest_rapid_date  rapid_win  rapid_draw  rapid_loss  current_blitz_rating  highest_blitz_rating highest_blitz_date  blitz_win  blitz_draw  blitz_loss  current_bullet_rating  highest_bullet_rating highest_bullet_date  bullet_win  bullet_draw  bullet_loss  highest_tactics_rating highest_tactics_date  highest_puzzle_rush_score  is_streamer   status   league
0           0       Komodo Engine     komodochess     https://www.chess.com/member/KomodoChess  24944922    GM  3411.0   United States        494  07/11/2015  17/09/2018                   NaN                   NaN                NaN        NaN         NaN         NaN                   NaN                   NaN                NaN        NaN         NaN         NaN                    NaN                    NaN                

In [6]:
dbpedia = SPARQLWrapper.SPARQLWrapper('https://dbpedia.org/sparql')


def queryDBpedia(query):
  dbpedia.setQuery(query)
  dbpedia.setMethod(SPARQLWrapper.POST)
  dbpedia.setReturnFormat(SPARQLWrapper.JSON)
  return dbpedia.query().convert()


def getChessInfoQueryString():
  return f"""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    SELECT ?birthName ?birthPlace ?url ?name WHERE {{
      ?chess a dbo:ChessPlayer ;
          foaf:isPrimaryTopicOf ?url ;
          dbp:birthPlace ?birthPlace ;
          dbp:birthName ?birthName ;
          foaf:name ?name.

    FILTER (lang(?birthPlace) = "en")
    FILTER (lang(?birthName) = "en")

    }}
    """


Wpisując imie i nazwisko martwego zawodnika, większość wartości jest NaN, przykłady "Garry Kasparov" - martwy, "David Howell" - żywy

In [7]:
name_input = 'David Howell, Jorge Cori, Fabiano Caruana' #Nie znajduje wszystkich osób (?) Na przykład Magnus Carlsen, czy Fabiano Caruana
name_input = [s.strip() for s in name_input.split(',')]

ch_df=chess_df[(chess_df['name']).isin(name_input)]
ch_df.rename(columns={'profile': 'url'}, inplace=True)

ch_df = ch_df.drop(['rapid_win', 'rapid_draw','rapid_loss','highest_puzzle_rush_score','highest_tactics_rating','highest_tactics_date'], axis='columns')
resInfo = queryDBpedia(getChessInfoQueryString())

birthNames = [res['birthName']['value'] for res in resInfo['results']['bindings']]
birthPlaces = [res['birthPlace']['value'] for res in resInfo['results']['bindings']]
urls = [res['url']['value'] for res in resInfo['results']['bindings']]
Name = [res['name']['value'] for res in resInfo['results']['bindings']]


temp_df = pd.DataFrame({'url': urls, 'birth name': birthNames, 'birth place': birthPlaces, "name": Name})
ch_df = ch_df.merge(temp_df, on='name')


display(ch_df)

<ipython-input-7-cdd46f29308b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ch_df.rename(columns={'profile': 'url'}, inplace=True)


,Unnamed: 0,name,username,url_x,user_id,title,fide,country,followers,joined,last_online,current_rapid_rating,highest_rapid_rating,highest_rapid_date,current_blitz_rating,highest_blitz_rating,highest_blitz_date,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,highest_bullet_date,bullet_win,bullet_draw,bullet_loss,is_streamer,status,league,url_y,birth name,birth place
0,67,David Howell,howitzer14,https://www.chess.com/member/howitzer14,28029696,GM,2689.0,England,966,04/05/2016,13/07/2023,2735.0,2906.0,17/08/2020,3019.0,3046.0,03/03/2021,3283.0,528.0,1009.0,3032.0,3032.0,09/09/2021,3818.0,324.0,1276.0,False,premium,Champion,http://en.wikipedia.org/wiki/David_Howell_(che...,David Wei Liang Howell,"Eastbourne, England, United Kingdom"
1,1268,Jorge Cori,machupichu10,https://www.chess.com/member/machupichu10,37634306,GM,NaN,Peru,621,08/08/2017,16/07/2023,2543.0,2835.0,23/08/2020,2760.0,2953.0,29/08/2021,566.0,95.0,413.0,3003.0,3020.0,02/04/2021,808.0,106.0,586.0,False,premium,Silver,http://en.wikipedia.org/wiki/Jorge_Cori,Jorge Moisés Cori Tello,"Lima, Peru"


In [8]:
temp_df.head()

,url,birth name,birth place,name
0,http://en.wikipedia.org/wiki/David_Howell_(che...,David Wei Liang Howell,"Eastbourne, England, United Kingdom",David Howell
1,http://en.wikipedia.org/wiki/David_Paravyan,David Arturovich Paravyan,"Moscow, Russia",David Paravyan
2,http://en.wikipedia.org/wiki/Jorge_Cori,Jorge Moisés Cori Tello,"Lima, Peru",Jorge Cori
3,http://en.wikipedia.org/wiki/Juan_Eduardo_Cast...,Juan Eduardo Castro dos Santos,"Montevideo,",J. Eduardo Castro
4,http://en.wikipedia.org/wiki/Juan_Eduardo_Cast...,Juan Eduardo Castro dos Santos,"Montevideo,",Kensei (拳生)
